In [18]:
import pandas as pd
import numpy as np
from operator import itemgetter
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
data = pd.read_csv('data/round_train.csv')

In [4]:
data = data[data.Tournament_Year==2003]

In [5]:
data.shape

(19393, 12)

In [6]:
data.head()

,Tournament_Year,Tournament_#,Permanent_Tournament_#,Course_#,Player_Number,Player_Name,Round_Number,Tee_Time,Round_Score,End_of_Event_Pos._(text),Finishing_Pos,Finishing_Pct
0,2003,10,16,656,1320,"Estes, Bob",1,11:50,66,T6,7.5,0.197917
1,2003,10,16,656,1361,"Forsman, Dan",1,11:00,69,T21,22.0,0.600694
2,2003,10,16,656,1761,"Mattiace, Len",1,13:00,75,24,24.0,0.656250
3,2003,10,16,656,1797,"Mediate, Rocco",1,12:10,72,T2,2.5,0.059028
4,2003,10,16,656,1920,"Parry, Craig",1,11:20,71,T25,26.5,0.725694


In [8]:
def label_players(player_inds,players):
    n = 0
    for player in players:
        if player not in player_inds:
            player_inds[player] = n
            n += 1
    return player_inds

In [9]:
player_inds = {}
player_inds = label_players(player_inds,data.Player_Number.values)
player_inds_reverse = dict((v, k) for k, v in player_inds.iteritems())

In [11]:
data.insert(5,'Index_Number',list(itemgetter(*data.Player_Number.values.tolist())(player_inds)))

In [13]:
rounds = data.groupby(['Tournament_Year','Tournament_#','Round_Number'])

In [14]:
len(rounds)

186

In [15]:
len(pd.unique(data.Player_Number))

665

In [17]:
data[data.Player_Name=='Woods, Tiger'].head()

,Tournament_Year,Tournament_#,Permanent_Tournament_#,Course_#,Player_Number,Index_Number,Player_Name,Round_Number,Tee_Time,Round_Score,End_of_Event_Pos._(text),Finishing_Pos,Finishing_Pct
2226,2003,60,4,104,8793,277,"Woods, Tiger",1,10:00,70,1,1.0,0.005531
2378,2003,60,4,4,8793,277,"Woods, Tiger",2,08:57,66,1,1.0,0.005531
2496,2003,60,4,4,8793,277,"Woods, Tiger",3,09:50,68,1,1.0,0.005531
2570,2003,60,4,4,8793,277,"Woods, Tiger",4,10:35,68,1,1.0,0.005531
2682,2003,70,7,500,8793,277,"Woods, Tiger",1,12:37,72,T5,5.5,0.047454


In [19]:
def fullprint(*args, **kwargs):
  from pprint import pprint
  opt = np.get_printoptions()
  np.set_printoptions(threshold='nan')
  pprint(*args, **kwargs)
  np.set_printoptions(**opt)

In [20]:
n_players = len(pd.unique(data.Player_Number))
n_rounds = len(rounds)

In [31]:
A = np.zeros((n_players,n_rounds,n_players))
G = np.zeros((n_players,n_rounds))
for u,df in enumerate(rounds):
    df = df[1]
#     if u>600: continue
    df = df.sort_values('Round_Score')
    n = len(df)
    G[df.Index_Number.values,u] = 1
    for row_ind in xrange(n):
        g_id = df.iloc[row_ind].Index_Number
        arr = np.zeros(n)
        arr[row_ind+1:] = 1
        A[g_id,u,df.Index_Number.values] += arr / n
    #fig, ax = plt.subplots(figsize=(20, 10))
    #plt.imshow(A[0,:,:])
    #plt.show()
    #fullprint(A[0,0:10,0:5])
    V = np.ndarray((n_players,n_players))
    for p in df.Index_Number:
        weights = np.matrix(G[p,:] / G.sum(1)[p])
        V[p,:] = (weights * A[p,:,:]).A1
    #fullprint(V[0:4,0:4])

In [32]:
p_nums = V.sum(1).argsort().tolist()[::-1][:40]
for u,p in enumerate(p_nums):
    if G.sum(1)[p] > 6:
        print data[data['Index_Number']==p].iloc[0].Player_Name,np.sort(V.sum(1)).tolist()[::-1][u],G.sum(1)[p]

Singh, Vijay 0.745916983546 102.0
Woods, Tiger 0.740639697103 68.0
Furyk, Jim 0.707500188855 101.0
Weir, Mike 0.703749028832 75.0
Els, Ernie 0.69105553549 60.0
Goosen, Retief 0.681931921547 64.0
Love III, Davis 0.671115365461 79.0
Evans, Gary 0.657844790824 8.0
Harrington, Padraig 0.647791927172 36.0
Campbell, Chad 0.635498034465 102.0
Couples, Fred 0.632137028893 67.0
Perry, Kenny 0.631856593107 97.0
Baird, Briny 0.617263064523 107.0
Allenby, Robert 0.61647716065 79.0
Cink, Stewart 0.616422999423 100.0
Jacobson, Freddie 0.614147070174 30.0
